# Europarts

## Инициализация

In [5]:
import os
import sys

from django.utils import timezone

sys.path.append('/home/ubuntu/anodos.ru/anodos/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'anodos.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


import re
import time

import catalog.runner
from catalog.models import *


class Runner(catalog.runner.Runner):

    name = 'EuroParts'
    alias = 'europarts'
    url = {'start' : 'http://euro-parts.ru/catalog/index.aspx',
           'base'  : 'http://euro-parts.ru',
           'price' : 'http://euro-parts.ru/catalog/index.aspx'}

    def __init__(self):

        super().__init__()

        self.stock = self.take_stock('stock', 'склад', 3, 10)
        self.transit = self.take_stock('transit', 'транзит', 10, 60)
        
    def run(self):

        # Заходим на начальную страницу
        tree = self.load_html(self.url['price'])

        # Проходим по всем категориям
        cs = tree.xpath('//div[@id="categories"]//ul[@class="list"]/li/a')
        for n, c in enumerate(cs):

            # Ждем, чтобы не получить отбой сервера
            time.sleep(1)

            category = str(c.text)
            print(category)

            # Загружаем список моделей
            c_url = self.xpath_string(c, './@href')
            c_url = '{}{}'.format(self.url['base'], c_url)
            print(c_url)
            tree = self.load_html(c_url)

            # Проходим по всем моделям
            ms = tree.xpath('//div[@class="catalog-list"]//a/@href')

            print('Моделей:', len(ms))

            if not len(ms):
                self.parse(tree, category)
                continue

            for m_url in ms:

                # Ждем, чтобы не получить отбой сервера
                time.sleep(1)

                m_url = '{}{}'.format(self.url['base'], m_url)
                print(m_url)
                tree = self.load_html(m_url)

                self.parse(tree, category)

        # Чистим устаревшие партии
        Party.objects.clear(stock = self.stock, time = self.start_time)

        # Пишем результат в лог
        self.log()

    def parse(self, tree, category):

        # Проходим по всем строкам
        rows = tree.xpath('.//div[@class="rows"]/ul[@class="row"]')
        
        print('Продуктов:', len(rows))

In [6]:
s = Runner()
s.run()

Цветные лазерные принтеры
http://euro-parts.ru/catalog/hp/2/index.aspx
Моделей: 156
http://euro-parts.ru/catalog/hp/2/4514/index.aspx?2851394670
Продуктов: 9
http://euro-parts.ru/catalog/hp/2/4515/index.aspx?2851394670
Продуктов: 3
http://euro-parts.ru/catalog/hp/2/4516/index.aspx?2851394670
Продуктов: 13
http://euro-parts.ru/catalog/hp/2/4517/index.aspx?2851394670
Продуктов: 6
http://euro-parts.ru/catalog/hp/2/4518/index.aspx?2851394670
Продуктов: 4
http://euro-parts.ru/catalog/hp/2/4519/index.aspx?2851394670
Продуктов: 4
http://euro-parts.ru/catalog/hp/2/8853/index.aspx?2851394670
Продуктов: 1
http://euro-parts.ru/catalog/hp/2/4520/index.aspx?2851394670
Продуктов: 8
http://euro-parts.ru/catalog/hp/2/4521/index.aspx?2851394670
Продуктов: 12
http://euro-parts.ru/catalog/hp/2/4522/index.aspx?2851394670
Продуктов: 11
http://euro-parts.ru/catalog/hp/2/4523/index.aspx?2851394670
Продуктов: 7
http://euro-parts.ru/catalog/hp/2/4524/index.aspx?2851394670
Продуктов: 7
http://euro-parts.ru/cata

Продуктов: 7
http://euro-parts.ru/catalog/hp/2/9603/index.aspx?2851394670
Продуктов: 7
http://euro-parts.ru/catalog/hp/2/9602/index.aspx?2851394670
Продуктов: 7
http://euro-parts.ru/catalog/hp/2/8493/index.aspx?2851394670
Продуктов: 8
http://euro-parts.ru/catalog/hp/2/9598/index.aspx?2851394670
Продуктов: 10
http://euro-parts.ru/catalog/hp/2/9599/index.aspx?2851394670
Продуктов: 10
http://euro-parts.ru/catalog/hp/2/6921/index.aspx?2851394670
Продуктов: 3
http://euro-parts.ru/catalog/hp/2/6939/index.aspx?2851394670
Продуктов: 1
http://euro-parts.ru/catalog/hp/2/8382/index.aspx?2851394670
Продуктов: 3
http://euro-parts.ru/catalog/hp/2/8835/index.aspx?2851394670
Продуктов: 5
http://euro-parts.ru/catalog/hp/2/10238/index.aspx?2851394670
Продуктов: 1
http://euro-parts.ru/catalog/hp/2/10239/index.aspx?2851394670
Продуктов: 1
http://euro-parts.ru/catalog/hp/2/10240/index.aspx?2851394670
Продуктов: 1
http://euro-parts.ru/catalog/hp/2/10241/index.aspx?2851394670
Продуктов: 1
http://euro-parts.r

KeyboardInterrupt: 